In [69]:
ts[['NAF 732',  'INTITULÉS 732', 'activiteEtablissement']]

,NAF 732,INTITULÉS 732,activiteEtablissement
0,2599A,Fabrication d'articles métalliques ménagers,16.21Z
395,1623Z,Fabrication de charpentes et d'autres menuiseries,16.23Z
396,1623Z,Fabrication de charpentes et d'autres menuiseries,16.23Z
397,1623Z,Fabrication de charpentes et d'autres menuiseries,16.23Z
398,1623Z,Fabrication de charpentes et d'autres menuiseries,16.23Z
...,...,...,...
2464,1623Z,Fabrication de charpentes et d'autres menuiseries,16.23Z
2465,1623Z,Fabrication de charpentes et d'autres menuiseries,16.23Z
2462,1621Z,Fabrication de placage et de panneaux de bois,16.21Z
2463,1621Z,Fabrication de placage et de panneaux de bois,16.21Z


In [74]:
import pandas as pd
ts1 = pd.read_json("../data/a7fb4f0f659a32eb35406c20111f5047.data", orient='index')
ts2 = pd.read_json("../data/570b01ab3c6ef9277b66b1aed4165ff2.data", orient='index')
ts=ts2
ts.columns
import natsort as ns

ts['groupEffectifEtablissement'] = pd.Categorical(ts['groupEffectifEtablissement'], ordered=True, categories= ns.natsorted(ts['groupEffectifUnite'].unique()))
ts = ts.sort_values('groupEffectifEtablissement')
#ts.columns

In [77]:
byRegion=pd.pivot_table(ts,
    index=['INTITULÉS 732'],
    aggfunc='count',
    values=['siret'],
    margins=True, margins_name = 'Total')

byRegion


,siret
INTITULÉS 732,
Fabrication d'emballages en bois,734
"Fabrication d'objets divers en bois - fabrication d'objets en liège, vannerie et sparterie",391
Fabrication de charpentes et d'autres menuiseries,1249
Fabrication de parquets assemblés,14
Fabrication de placage et de panneaux de bois,79
Total,2467


In [72]:
pd.crosstab([ts.activiteEtablissement],ts.region, rownames=['Activité'], colnames=['Régions'])

Régions,Auvergne-Rhône-Alpes,Bourgogne-Franche-Comté,Bretagne,Centre-Val de Loire,Corse,Grand Est,Hauts-de-France,Normandie,Nouvelle-Aquitaine,Occitanie,Outre-mer,Pays de la Loire,Provence-Alpes-Côte d'Azur,Île-de-France
Activité,,,,,,,,,,,,,,
16.10A,237,174,28,48,2,163,34,46,224,95,20,30,22,2


In [47]:
pd.crosstab([ts.groupEffectifEtablissement],ts.groupCA, rownames=['Effectif'], colnames=['Chiffre d affaire'],margins=True)

Chiffre d affaire,1-5 M€,10-50 M€,100-500 k€,5-10 M€,500-1000 k€,<100 k€,>50 M€,Inconnu,All
Effectif,,,,,,,,,
1 ou 2 salariés,3,0,39,1,5,4,0,262,314
3 à 5 salariés,13,0,28,0,26,0,0,286,353
6 à 9 salariés,39,0,9,0,36,2,0,207,293
10 à 19 salariés,74,1,0,9,8,0,0,240,332
20 à 49 salariés,33,15,0,29,0,0,0,89,166
50 à 99 salariés,0,17,0,2,0,0,0,14,33
100 à 199 salariés,0,1,0,0,0,0,0,1,2
200 à 249 salariés,0,0,0,0,0,0,2,0,2
250 à 499 salariés,0,0,0,0,0,0,0,1,1


In [48]:
tsg = ts.groupby('groupEffectifEtablissement').agg({'CAAverageCorrected': ['mean'], "siren": "count"})
pd.options.display.float_format = '{:,.1f} M€'.format
tsg['Chiffre d affaire']=tsg[('CAAverageCorrected',  'mean')]*tsg[('siren',  'count')]
tsg.columns = ['_'.join(col) for col in tsg.columns.values]
tsg = tsg.reset_index()
tsg.rename(columns={'groupEffectifEtablissement': 'Effectif', 
                    'CAAverageCorrected_mean':  "Chiffre d'affaire moyen", 
                    'siren_count':"Nombre d'entreprises", 
                    "Chiffre d affaire_":"Chiffre d'affaire Total"}, inplace=True)
import extract
for index, row in tsg.iterrows():
    tsg.at[index, 'valeurEffectif']= extract.groupEffectifToValeurEffectif(row['Effectif'])
tsg['Effectif Total']=tsg['valeurEffectif'].astype(int)*tsg["Nombre d'entreprises"].astype(int)
tsg[['Effectif',"Chiffre d'affaire moyen", "Nombre d'entreprises", "Chiffre d'affaire Total" ,"Effectif Total"  ]]

,Effectif,Chiffre d'affaire moyen,Nombre d'entreprises,Chiffre d'affaire Total,Effectif Total
0,1 ou 2 salariés,0.5 M€,314,144.3 M€,314
1,3 à 5 salariés,0.7 M€,353,256.7 M€,1412
2,6 à 9 salariés,1.1 M€,293,329.9 M€,2051
3,10 à 19 salariés,2.6 M€,332,877.1 M€,4980
4,20 à 49 salariés,7.1 M€,166,"1,176.8 M€",5810
5,50 à 99 salariés,17.5 M€,33,577.8 M€,2475
6,100 à 199 salariés,42.5 M€,2,84.9 M€,400
7,200 à 249 salariés,71.7 M€,2,143.4 M€,450
8,250 à 499 salariés,nan M€,1,nan M€,375


In [49]:
tsg.aggregate({
              "Nombre d'entreprises":['sum'], 
              "Chiffre d'affaire Total":['sum'],
    'Effectif Total':['sum'],
              }) 

,Nombre d'entreprises,Chiffre d'affaire Total,Effectif Total
sum,1496,"3,591.0 M€",18267


In [ ]:
s1=list(ts1[ts1["siren"] == 315620104]["search"])
print(s1)

In [ ]:
s2=ts2[ts2["siren"] == 315620104]["search"]
print(s2)

In [ ]:
for i in s1:
    print(i)
    for ii in i:
        print(ii)

In [ ]:
for i in s2:
    print(i)
    for ii in i:
        print(ii)

In [ ]:
bufferSearch = pd.read_csv("../data/bufferSearch.csv")
from ast import literal_eval
bufferSearch['search'] = df['search'].apply(literal_eval)

buf=bufferSearch.loc[bufferSearch['denomination'] == "ETABLISSEMENTS BORIE", 'search']
r=buf.iloc[0]
print (r)
type(r)

In [ ]:
from googlesearch import search
r=search("ETABLISSEMENTS BORIE")
print(r)
type(r)

In [ ]:
for idx, word in enumerate(s):
    print (idx, word)

In [ ]:
ls=list(s)
ls

In [ ]:
for index, row in df.iterrows():
    if row["siren"] not in sirens:
        ld=ld+row["domains"]
        
    sirens.append(row["siren"])

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
df[["domains", "search"]]

In [ ]:
import tldextract

def domains(row):
    ld = set()    
    for item in row["search"]:
        print (item)
        d = tldextract.extract(item)
        dom = '.'.join(d[1:3])
        ld.add(dom)
    return ld

In [ ]:
import logging
import extract
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

logging.basicConfig(format='%(asctime)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

params = {"codeCommuneEtablissement": "382*","etatAdministratifUniteLegale": "A",
'categorieJuridiqueUniteLegale': "5*",
'activitePrincipaleUniteLegale': "16.10A", 'etablissementSiege':True}
file =extract.query(logger, params)

import myGoogleSearch
ts=myGoogleSearch.go(file, logger)

In [ ]:
ts.columns

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


(ts[["denomination", 'scoreOwnedDomain',"statusDomain","ownedDomain" ,"ownedDomain2", "ownedDomain3"]])

In [ ]:
ts["statusDomain"].tolist

In [ ]:
def statusDomain(row):
    d=[row['ownedDomain'], row['ownedDomain2'], row['ownedDomain3']]
    dnonone=list(filter(lambda a: a !="NONE", d))
    nb=len(list(dict.fromkeys(d)))
    nbnonone=len(list(dict.fromkeys(dnonone)))
    if nbnonone ==0:
        return "NONE"
    best=row['ownedDomain']
    bestScore=row['scoreOwnedDomain']
    if row['scoreOwnedDomain2'] > bestScore:
        best=row['ownedDomain2']
        bestScore=row['scoreOwnedDomain2']
    
    if row['scoreOwnedDomain3'] > bestScore:
        best=row['ownedDomain3']
        bestScore=row['scoreOwnedDomain3']
    return best

'''d=[
        [row['ownedDomain'],row['scoreOwnedDomain']],
        [row['ownedDomain2'],row['scoreOwnedDomain2']],
        [row['ownedDomain3'],row['scoreOwnedDomain3']]
      ]
'''
ts['statusDomain'] = ts.apply(statusDomain, axis=1)

ts.groupby(['statusDomain'])[['siret']].count()

In [ ]:
len(ts[ts['searchTitle'] == '-'])

In [ ]:
ts.groupby(['effectif'])[['siret']].count()

In [ ]:
import pandas as pd
nafs=pd.read_csv("nafs.csv",sep=";")


In [ ]:
ts_naf=pd.merge (ts,nafs)
ts_naf.columns

In [ ]:
ts_naf.groupby(['departement','INTITULÉS 732'])[['siret']].count()

In [ ]:
import pandas as pd

finance=pd.read_csv("chiffres-cles-2019_38.csv",sep=";")
finance.rename(columns={'Siren':'siren'
                                }, inplace=True)
finance = finance.astype({"siren":  int})
ts_naf = ts_naf.astype({"siren":  int})
finance.columns

In [ ]:
ts_naf_fi=pd.merge(ts_naf, finance[['siren','CA 1','CA 2', 'CA 3']], on='siren', how='left')


In [ ]:
ts_naf_fi

In [ ]:
import numpy as np

pd.pivot_table(ts_naf_fi,index=['INTITULÉS 732'],columns=['departement'],values=["CA 1", "CA 2", "CA 3"],aggfunc=[len, np.mean, np.sum]) .style.format('{0:,.0f}')

In [ ]:
pd.pivot_table(ts_naf_fi,index=['INTITULÉS 732'],columns=['departement'],values=["siret"],aggfunc=[len] ,margins=True)

In [ ]:
ts_naf_fi.isnull().sum(axis = 0)


In [ ]:
ts_naf_fi[ts_naf_fi.isna().any(axis=1)]


In [ ]:
from googlesearch import search
search("BOIS DU DAUPHINE")


import lxml.html
t = lxml.html.parse("https://www.dnb.com/business-directory/company-profiles.provost_freres.66acf47c517a112c017d56760ecf141f.html")
print (t.find(".//title").text)

In [ ]:
# importing the modules 
import requests 
from bs4 import BeautifulSoup 
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}

# target url 
url2 = "https://www.dnb.com/business-directory/company-profiles.provost_freres.66acf47c517a112c017d56760ecf141f.html"
url = "https://aurora-5r.fr"
print("kkk")

# making requests instance 
reqs = requests.get(url3,  headers=header) 
print("ddd")
# using the BeaitifulSoup module 
soup = BeautifulSoup(reqs.text, 'html.parser') 
  
# displaying the title 
print("Title of the website is : ") 
for title in soup.find_all('title'): 
    print(title.get_text())

In [ ]:
from urllib.request import urlopen, Request
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                      'AppleWebKit/537.11 (KHTML, like Gecko) '
                      'Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}


reg_url = url3
req = Request(url=reg_url, headers=headers) 
html = urlopen(req).read() 
print(html)

In [ ]:

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
html = urllib.request.urlopen(url2, context=ctx).read()
html

In [ ]:
url3="https://www.scieriesanguinet.fr/"

def url_status(url):
       headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0)'
                                ' Gecko/20100101 Firefox/24.0'}
       REQUEST_TIMEOUT = 10

       if 'http' not in url:
           url = 'http://' + url
       try:
           response = requests.get(url, headers=headers, timeout=REQUEST_TIMEOUT)
           if(response.status_code != 200):
               print(url)
               print('status',response.status_code)
           return response.status_code
       except Exception as e:
           print(url)
           print('Error',e)
           return -1
url_status(url3)

In [ ]:
import requests
import eventlet
eventlet.monkey_patch()

with eventlet.Timeout(10):
    requests.get("http://ipv4.download.thinkbroadband.com/1GB.zip", verify=False)